In [ ]:
### Install all dependecies

# pytorch3d
import os
import sys
import torch

need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True

if need_pytorch3d:
    if torch.__version__.startswith("1.10.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'


# cleanup
!rm -rf 1.10.0.tar.gz cub-1.10.0/

In [ ]:
import os
import sys
import torch

import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.io import read_image
from typing import Union

from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes

from pytorch3d.renderer import (
    PerspectiveCameras,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    AmbientLights,
    BlendParams,
    TexturesUV
)

In [ ]:
### Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
### Construct textured mesh for a specific subject and pose

def construct_mesh(subject:int, pose:str, default_mesh_path:str):
    default_mesh = load_obj(default_mesh_path, load_textures=False)

    verts_uvs = default_mesh[2].verts_uvs.to(device).unsqueeze(0)
    faces_uvs = default_mesh[1].textures_idx.to(device).unsqueeze(0)

    texture_filename = 'subject_%d/body/%s/appearance/median_map.png' % (subject, pose)
    texture_img = read_image(texture_filename)
    texture_img = torch.moveaxis(texture_img, 0, 2).unsqueeze(0).float() * 1.0/255
    texture_uv = TexturesUV(maps=texture_img, faces_uvs=faces_uvs, verts_uvs=verts_uvs)

    verts_filename = 'subject_%d/body/%s/reconstruction/smpl_vertex.txt' % (subject, pose)
    mesh_verts = torch.Tensor( np.loadtxt(verts_filename) ).to(device).unsqueeze(0)
    mesh_faces = default_mesh[1].verts_idx.to(device).unsqueeze(0)

    mesh = Meshes(mesh_verts, mesh_faces, texture_uv)

    return mesh

In [ ]:
### Extract camera calibration parameters for a subject
# Returns the camera ids and the corresponding parameters

def camera_calibration(subject:int, calibration:str='intrinsic'):
    assert( calibration in ['intrinsic', 'extrinsic', 'project'] ), "calibration needs to be one of 'intrinsic', 'extrinsic' or 'project'"

    filename = 'subject_%d/body/%s.txt' % (subject, calibration)

    with open(filename) as f:
        lines = f.read().splitlines()[3:]

    if calibration == 'intrinsic':
        rows = 3
        cols = 3
    elif calibration == 'extrinsic':
        rows = 4
        cols = 3
    elif calibration == 'project':
        rows = 3
        cols = 4

    camera_id = [int(id[1:]) for id in lines[::rows+1]]

    parameters = lines.copy()
    del parameters[0::rows+1]
    parameters = torch.from_numpy(np.loadtxt(parameters))
    parameters = parameters.reshape(len(camera_id), rows, cols)

    return camera_id, parameters

In [ ]:
### Extract camera matrices for a subject and camera

def get_camera_parameters(subject:int, camera_index:int):
    assert(camera_index in range(107)), 'camera_index must be an integer in between 0 and 106'
    intr_id, intrinsic = camera_calibration(subject, calibration='intrinsic')
    extr_id, extrinsic = camera_calibration(subject, calibration='extrinsic')
    proj_id, projection = camera_calibration(subject, calibration='project')
    # intr_id = extr_id = proj_id

    cam = intr_id.index(camera_index)

    C = extrinsic[cam][0]
    R = extrinsic[cam][1:]
    T = - torch.matmul(R, C)
    K = intrinsic[cam]

    R = R.T.unsqueeze(0)
    T = T.unsqueeze(0)

    f = torch.tensor((K[0,0], K[1,1]), dtype=torch.float32).unsqueeze(0)
    p = torch.tensor((K[0,2], K[1,2]), dtype=torch.float32).unsqueeze(0)
    
    return R, T, f, p

In [ ]:
### Get renderer

def get_renderer(cameras:pytorch3d.renderer.cameras, image_size:Union[int, tuple[int, int]]=(360, 640), device=device):
    raster_settings = RasterizationSettings(
        image_size=image_size,
        blur_radius=0.0,
        faces_per_pixel=1,
        max_faces_per_bin=10000
    )

    blend_params = BlendParams(background_color=(0, 0, 0))
    lights = AmbientLights(device=device)

    renderer = MeshRenderer(
        rasterizer=MeshRasterizer(
            cameras=cameras, 
            raster_settings=raster_settings
        ),
        shader=SoftPhongShader(
            device=device, 
            cameras=cameras,
            lights=lights,
            blend_params=blend_params
        )
    )

    return renderer

In [ ]:
subject = 1
pose = '00000001'
obj_mesh = 'smplx/text_uv_coor_smpl.obj'

humbi_smpl_mesh = construct_mesh(subject, pose, obj_mesh)

camera_idx = 73 # camera view to render
R, T, f, p = get_camera_parameters(subject, camera_idx)

img_size = (1080, 1920)
cameras = PerspectiveCameras(focal_length=-f, principal_point=p, R=R, T=T, in_ndc=False, image_size=(img_size,), device=device)

render_img_size = (270, 480) # render resolution
renderer = get_renderer(cameras, render_img_size, device)
render = renderer(humbi_smpl_mesh).detach()
rgb_render = render[0, ..., :3]

photo_path = 'subject_%s/body/%s/image/image%s.jpg' % (subject, pose, str(camera_idx).zfill(7))
photo = read_image(photo_path)

rescale = transforms.Resize(render_img_size)
resized_photo = rescale(photo).permute(1, 2, 0)

In [ ]:
plt.figure(figsize=(16, 9))
plt.imshow(rgb_render.cpu().numpy())
plt.axis("off")

In [ ]:
plt.figure(figsize=(16, 9))
plt.imshow(resized_photo.cpu().numpy())
plt.axis("off")